## 01. Importing Libraries & Data

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os

In [3]:
# Import data set
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_derived.pkl'))

In [2]:
# Create Path
path = r'C:\Users\Dell\Documents\Instacart Basket Analysis'

In [4]:
# Dropping the "aisle_id" column from the ords_prods_merge dataframe
ords_prods_merge.drop(columns = ['aisle_id'])

,product_id,product_name,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day
0,1,Chocolate Sandwich Cookies,19,5.8,3139998,138,28,6,11,3.0,5,0,both,Mid-range product,Regularly busy,Regularly busy,Most orders
1,1,Chocolate Sandwich Cookies,19,5.8,1977647,138,30,6,17,20.0,1,1,both,Mid-range product,Regularly busy,Regularly busy,Average orders
2,1,Chocolate Sandwich Cookies,19,5.8,389851,709,2,0,21,6.0,20,0,both,Mid-range product,Busiest day,Busiest days,Average orders
3,1,Chocolate Sandwich Cookies,19,5.8,652770,764,1,3,13,NaN,10,0,both,Mid-range product,Regularly busy,Slowest days,Most orders
4,1,Chocolate Sandwich Cookies,19,5.8,1813452,764,3,4,17,9.0,11,1,both,Mid-range product,Least busy,Slowest days,Average orders
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32404854,49688,Fresh Foaming Cleanser,11,13.5,1788356,200215,2,0,9,5.0,27,0,both,Mid-range product,Busiest day,Busiest days,Most orders
32404855,49688,Fresh Foaming Cleanser,11,13.5,3401313,200377,1,4,11,NaN,5,0,both,Mid-range product,Least busy,Slowest days,Most orders
32404856,49688,Fresh Foaming Cleanser,11,13.5,809510,200873,5,3,8,15.0,12,0,both,Mid-range product,Regularly busy,Slowest days,Average orders
32404857,49688,Fresh Foaming Cleanser,11,13.5,2359893,200873,9,3,15,5.0,11,1,both,Mid-range product,Regularly busy,Slowest days,Most orders


## 02. Calculate the Mean of Order Number

In [ ]:
# Use groupby() and agg() functions to calculate mean of user_order_number
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

## 03. Analyze the result. How do the results for the entire dataframe differ from those of the subset? 

#### 
When you compare the mean order numbers for the entire dataset with those of a subset, the most apparent observation is the increased number of departments listed. This suggests that the overall mean order numbers vary across departments when considering the entire dataset due to differences in the count of department IDs

## 04. Create Loyalty Flag for Exisiting Customers

In [ ]:
# Use groupby and transform() functions with np.max argument to determine maximum order number for each user
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [ ]:
# View updated data frame
ords_prods_merge.head(20)

In [ ]:
# Use loc function to make loyalty flag column with given criteria
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [ ]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [ ]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [ ]:
# Check frequency of values in new loyalty flag column
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

In [ ]:
# View updated data frame
ords_prods_merge.head()

## 05. finding the spending habits of loyality group

In [ ]:
# Use groupby and agg() to determine basic statisitcs of spending habits by loyalty group
ords_prods_merge.groupby('loyalty_flag').agg({'prices' : ['mean', 'min', 'max']})

#### The average price of a product bought by a loyal customer is lower than the price of a product purchased by both regular and new customers.

## 06.  Creating a spending flag for each user based on the average price across all their orders ..”

In [ ]:
# Use transform() to determine the mean price of products purchased by each user
ords_prods_merge['mean_product_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [ ]:
# View the columns 
ords_prods_merge[['user_id', 'mean_product_price']].head(20)

In [ ]:
# Create spending_flag column with given criteria using loc() function
ords_prods_merge.loc[ords_prods_merge['mean_product_price'] >= 10, 'spending_flag'] = 'High spender'

In [ ]:
ords_prods_merge.loc[ords_prods_merge['mean_product_price'] < 10, 'spending_flag'] = 'Low spender'

In [ ]:
# Checking the new spending_flag column
ords_prods_merge['spending_flag'].value_counts(dropna = False)

In [ ]:
# View relevant columns of updated data frame
ords_prods_merge[['user_id', 'mean_product_price', 'spending_flag']].head(20)

## 07. Determination of frequent and non-frequent users

In [ ]:
# Use groupby and transform to determine median number of days between orders
ords_prods_merge['median_days_between_orders'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [ ]:
# Create order frequency flag based on given criteria using loc function
ords_prods_merge.loc[ords_prods_merge['median_days_between_orders'] > 20, 'order_frequency'] = 'Non-frequent customer'


In [ ]:
ords_prods_merge.loc[(ords_prods_merge['median_days_between_orders'] <= 20) & (ords_prods_merge['median_days_between_orders'] > 10), 'order_frequency'] = 'Regular customer'

In [ ]:
ords_prods_merge.loc[ords_prods_merge['median_days_between_orders'] <= 10, 'order_frequency'] = 'Frequent customer'

In [ ]:
# Checking the value counts of new order_frequency column
ords_prods_merge['order_frequency'].value_counts(dropna = False)

#### 
It appears that some rows were left unlabeled. One possible reason for this could be that certain users have made only a single order

In [ ]:
# View the columns of updated data frame
ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days_between_orders', 'order_frequency']].head(25)

## 09. Export the dataframe as a pickle file

In [ ]:
# Export derived data frame as pickle file
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_grouped.pkl'))